In [21]:
#!/usr/bin/env python3
"""
This script “discovers” a set of coefficients for the 8th–order Yoshida integrator.
We assume a symmetric (palindromic) composition of a basic second–order integrator
with time–step coefficients:
    S(Δt) = S(w7 Δt) S(w6 Δt) ... S(w1 Δt) S(w0 Δt) S(w1 Δt) ... S(w7 Δt),
with the normalization condition:
    w0 = 1 - 2*(w1 + w2 + ... + w7).

The order conditions (to cancel the error terms at O(Δt^3), O(Δt^5), and O(Δt^7)) are:
    w0^3 + 2 Σ_{i=1}^7 w_i^3 = 0,
    w0^5 + 2 Σ_{i=1}^7 w_i^5 = 0,
    w0^7 + 2 Σ_{i=1}^7 w_i^7 = 0.
We then compute the drift (c) and kick (d) coefficients as:
    c1 = 0.5 * w7,  c2 = 0.5*(w7+w6), ..., c8 = 0.5*(w1+w0),
with the kick coefficients given by:
    d1 = w7, d2 = w6, ..., d8 = w0.
"""

import numpy as np
from scipy.optimize import minimize

def objective(w_free):
    """
    w_free is an array [w1, w2, ..., w7].
    We compute w0 from normalization:
        w0 = 1 - 2*(w1 + ... + w7)
    and form the sum-of-squares of the error conditions:
        E = (w0^3 + 2 Σ w_i^3)^2 + (w0^5 + 2 Σ w_i^5)^2 + (w0^7 + 2 Σ w_i^7)^2.
    Our goal is to drive E as close to zero as possible.
    """
    S = np.sum(w_free)
    w0 = 1 - 2 * S
    eq3 = w0**3 + 2 * np.sum(w_free**3)
    eq5 = w0**5 + 2 * np.sum(w_free**5)
    eq7 = w0**7 + 2 * np.sum(w_free**7)
    return eq3**2 + eq5**2 + eq7**2

def objective_with_penalty(w_free):
    """
    Extended objective that also penalizes deviations from the published w3.
    """
    S = np.sum(w_free)
    w0 = 1 - 2 * S
    eq3 = w0**3 + 2 * np.sum(w_free**3)
    eq5 = w0**5 + 2 * np.sum(w_free**5)
    eq7 = w0**7 + 2 * np.sum(w_free**7)
    
    # Published value for w3 (the third free parameter, index 2) from Yoshida:
    target_w3 = -0.00716#989419708
    # A penalty coefficient (this number is somewhat arbitrary; you may need to adjust it)
    penalty_weight = 1000.0
    penalty = penalty_weight * (w_free[2] - target_w3)**2

    return eq3**2 + eq5**2 + eq7**2 + penalty


if __name__ == '__main__':
    # Initial guess taken (approximately) from one published solution:
    w_initial = np.array([
        -1.61582374150097,
        -2.44699182370524,
        -0.0716989941970812,
         2.44002732616735,
         0.1577399281236,
         1.8202063097071,
         1.0424262086999
    ])
    
    # Use Nelder-Mead with appropriate option names ('xatol' and 'fatol')
    res = minimize(objective, w_initial, method='Nelder-Mead',
                   options={'xatol': 1e-16, 'fatol': 1e-6, 'disp': True, 'maxfev' : 10000})
    
    # Extract the free parameters:
    w_free = res.x
    S = np.sum(w_free)
    w0 = 1 - 2 * S

    print("Found w coefficients:")
    print(f"w0 = {w0:.15g}")
    for i, wi in enumerate(w_free, start=1):
        print(f"w{i} = {wi:.15g}")
        
    # Print final objective value (should be near zero)
    final_obj = objective(w_free)
    print(f"\nFinal objective (error) = {final_obj:.3e}")
        
    # Compute drift (c) coefficients.
    # Palindromic ordering:
    # S(w7 dt) S(w6 dt) ... S(w1 dt) S(w0 dt) S(w1 dt) ... S(w7 dt)
    c = np.empty(8)
    c[0] = 0.5 * w_free[6]                       # c1 = w7/2
    c[1] = 0.5 * (w_free[6] + w_free[5])           # c2 = (w7+w6)/2
    c[2] = 0.5 * (w_free[5] + w_free[4])           # c3 = (w6+w5)/2
    c[3] = 0.5 * (w_free[4] + w_free[3])           # c4 = (w5+w4)/2
    c[4] = 0.5 * (w_free[3] + w_free[2])           # c5 = (w4+w3)/2
    c[5] = 0.5 * (w_free[2] + w_free[1])           # c6 = (w3+w2)/2
    c[6] = 0.5 * (w_free[1] + w_free[0])           # c7 = (w2+w1)/2
    c[7] = 0.5 * (w_free[0] + w0)                  # c8 = (w1+w0)/2
    
    # The kick (d) coefficients are:
    d = np.empty(8)
    d[0] = w_free[6]  # d1 = w7
    d[1] = w_free[5]  # d2 = w6
    d[2] = w_free[4]  # d3 = w5
    d[3] = w_free[3]  # d4 = w4
    d[4] = w_free[2]  # d5 = w3
    d[5] = w_free[1]  # d6 = w2
    d[6] = w_free[0]  # d7 = w1
    d[7] = w0         # d8 = w0
    
    # print("\nDrift (c) coefficients:")
    # for i, ci in enumerate(c, start=1):
    #     print(f"c{i} = {ci:.15g}")
        
    # print("\nKick (d) coefficients:")
    # for i, di in enumerate(d, start=1):
    #     print(f"d{i} = {di:.15g}")
        
    # Note: By symmetry, the full sequence of c coefficients is:
    # c1, c2, ..., c8, c9=c8, c10=c7, ..., c16=c1.


Found w coefficients:
w0 = -2.09713231474853
w1 = -1.77374280940536
w2 = -2.4807315296137
w3 = -0.0680868000530122
w4 = 2.46276718752474
w5 = 0.188705427483179
w6 = 2.08392408430459
w7 = 1.13573059713383

Final objective (error) = 1.265e-27


C:\Users\flori\AppData\Local\Temp\ipykernel_36752\625221395.py:71: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(objective, w_initial, method='Nelder-Mead',


    constexpr double w8_1 = -0.161582374150097E1;
    constexpr double w8_2 = -0.244699182370524E1;
    constexpr double w8_3 = -0.716989419708120E-2;
    constexpr double w8_4 = 0.244002732616735E1;
    constexpr double w8_5 = 0.1577399281236E0;
    constexpr double w8_6 = 0.18202063097071E1;
    constexpr double w8_7 = 0.10424262086999E1;
    constexpr double w8_0 = 1 - 2 * (w8_1 + w8_2 + w8_3 + w8_4 + w8_5 + w8_6 + w8_7);